# 4.3. Limpieza y preparación de datos.

In [ ]:
import pandas as pd
import numpy as np

## Tratamiento de datos en blanco (<i>missing values</i>)

- En la mayoría de los ficheros utilizados como fuente de datos, es muy común la existencia de valores nulos (en blanco, <i>missing</i>...). 
- Estos "huecos" en la información suelen ser muy problemáticos, ya que tiene un impacto importante a la hora de realizar cualquier tipo de cálculo numérico y son difícilmente interpretables.
- Uno de los objetivos de pandas es facilitar el tratamiento de este tipo de datos no existentes, ofreciendo múltiples funciones que permiten llevar a cabo tanto su detección, como su eliminación o imputación...

#### Detección de <i>missing values</i>

Pandas ofrece principalmente dos funciones para manejar la detección de valores nulos.<br/>
<ul>
<li><b>isnull:</b> Devuelve una Serie o DataFrame booleano indicando qué elemetos son NaN o None.</li>
<li><b>notnull:</b> Devuelve el inverso del anterior.</li>

Generamos una serie con un null

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

Obtenemos su índice booleano

In [ ]:
string_data.isnull()

Ahora ponemos el primer dato a None

In [ ]:
string_data[0] = None
string_data

Y volvemos a calcular el índice booleano

In [ ]:
string_data.isnull()

Como podemos ver, NaN, Null y None, son detectados por la función isnull()

#### Eliminación de registros con <i>missing values</i>

- Conviene hacer un estudio cuidadoso del por qué y la casuística de los valores nulos (especialmente en finanzas)
- Uno de los posibles tratamientos a aplicar es su eliminación directa del set de datos.
- Pandas, nos ofrece el método <b>dropna</b> para llevar a cabo esta tarea.
- Eliminando toda la fila o columna que tenga un missing value
- Los parámetros de este método son:<br/>
<ul>
<li><b>axis:</b> Selección de eje sobre el que realizar la eliminación.</li>
<li><b>how:</b> Tomará posibles valores 'any' y 'all' e indica si se debe eliminar cuando haya algún valor con NaN, o cuando todos los valores sean NaN.</li>
<li><b>thresh:</b> Permite indicar el número de observaciones no nulas que se deben tener para no realizar el borrado.</li>
</ul>

Generamos una serie de datos con None y NAs

In [ ]:
data = pd.Series([1, None, 3.5, np.nan, 7])
data

Eliminamos los datos marcados como NaN

In [ ]:
data.dropna()

Otra forma de hacerlo sería extraer aquellos datos considerados no nulos

In [ ]:
data[data.notnull()]

Generamos ahora un DF

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.],
                     [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan],
                     [np.nan, 6.5, 3.]])
data

La única fila sin NAs es la 1ª. POr lo que si aplicamos dropna, será la única fila que quede

In [ ]:
cleaned = data.dropna() # por defecto axis = 0
cleaned

In [ ]:
data

Podríamos especificar que queremos que nos borre únicamente las filas en las que todos los datos sean NaN

In [ ]:
data.dropna(how='all')

Añadimos una nueva columna al DF

In [ ]:
data[4] = np.nan
data

Y le indicamos que aplique el dropna por columnas, únicamente a aquellas en donde todos los datos sean NA

In [ ]:
data.dropna(axis=1, how='all')

Generamos un nuevo DF

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

Si aplicamos dropna, únicamente las 3 últimas filas sobrevivirán

In [ ]:
df.dropna()

Pero podemos decirle que elimine únicamente aquellas filas que tengan menos de dos datos válidos

In [ ]:
df.dropna(thresh=2)

En ese caso, las filas 2 y 3 permanecen

#### Relleno de registros con <i>missing values</i>

- Existirán casos en los que no se desee (o no se pueda) eliminar los registros con valores nulos (p.e. podrían suponer un porcentaje demasiado elevado de nuestro set de datos). 
- En estos casos, habrá que realizar una imputación de los mismos a un valor preestablecido.
- Para esta tarea Pandas incorpora el método <b>fillna</b>, que cuenta con los siguientes parámetros:<br/>
<ul>
<li><b>axis:</b> Que decide si aplicará el criterio de relleno por filas o columnas.</li>
<li><b>value:</b> Que rellena los valores nulos a un valor fijo.</li>
<li><b>method:</b> Que permitirá establecer un criterio de relleno de entre los siguientes:
<ul>
<li>ffill: Relleno en base a la observación de los últimos elementos no nulos.</li>
<li>bfill: Relleno en base a la observación de los próximos elementos no nulos.</li>
</ul>
<li><b>limit:</b> Contador máximo de elmentos imputados.</li>
</ul>

In [ ]:
df

Rellenamos los NaN a cero

In [ ]:
df.fillna(0)

Podemos hacer un dictionario con el valor de relleno para cada columna

In [ ]:
df.fillna({1: 0.5, 2: 0})

Podemos indicar que el relleno se realice inplace

In [ ]:
df.fillna(0, inplace=True)
df

Generamos un nuevo DF

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

Rellenamos en base a la observación de los últimos elementos no nulos

In [ ]:
df.fillna(method='ffill')

También podemo poner un límite a los datos que queremos rellenar con este método

In [ ]:
df.fillna(method='ffill', limit=2)

Calculamos la media por columnas

In [ ]:
df.mean()

Y aplicamos la media para realizar el relleno

In [ ]:
df.fillna(df.mean())

Podemos hacer lo mismo con una serie

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

In [ ]:
data.fillna(data.mean())

### Eliminación de duplicados

Generamos un DF con datos sobre los que trabajar

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

Duplicated devuelve un índice booleano con TRUE en el caso de que todos los datos de una fila estén duplicados (si hay una fila entera duplicada)

In [ ]:
data.duplicated()

En ese caso, podemos usar drop para eliminar la fila duplicada

In [ ]:
data.drop_duplicates()

Añadimos una nueva columna al DF

In [ ]:
data['v1'] = range(7)
data

Podemos indicar que aplique el drop sobre los datos de una única columna

En este caso, solo sobreviven las primeras observaciones, no duplicadas

In [ ]:
data.drop_duplicates(['k1'])

Con keep, podemos indicar que queremos que sobrevivan las últimas observaciones, no duplicadas

In [ ]:
data.drop_duplicates(['k1'], keep='last')

Este comportamiento podemos hacerlo tan complejo como queramos.

Elimina los duplicados de las columnas k1 y k2 (cuando toda la fila sea igual), manteniendo las últimas observaciones

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Remplazamiento de valores

Generamos los datos sobre los que trabajar

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

Queremos reemplazar el -999 por 10

In [ ]:
data.replace(-999, 10)

Podríamos querer convertir algunos datos en nan, para procesarlos después

In [ ]:
data.replace([-999, -1000], np.nan)

Podemos reemplazar más de un dato a la vez, por más de un sustituto.

En este caso convertimos los -999 en nan, y los -1000 en 0

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

Podríamos conseguir el mismo comportamiento pasándole un diccionario

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renombrado de índices y columnas

Para renombrar tanto el índice de filas (index), como el de columnas (columns), debemos cambiarlo entero

Generamos un DF sobre el que trabajar

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

Programamos una función lambda que convierte en mayúscula las dos primeras letras que recibe de cada elemento

In [ ]:
transform = lambda x: x[:2].upper()

Aplicamos la función al índice del DF (no lo estamos sobreescribiendo aún)

In [ ]:
data.index.map(transform)

Aplicamos la función para que sea el nuevo índice del DF

In [ ]:
data.index = data.index.map(transform)
data

También podemos decirle que el índice lo queremos como un título: la 1ª el mayúscula y el resto en minúscula

Y que el índice de las columnas esté en mayúscula

In [ ]:
data.rename(index=str.title, columns=str.upper)
data

Podemos renombrar filas y columnas diréctamente, especificando el nombre anterior y el nuevo

In [ ]:
data = data.rename(index={'OH': 'INDIANA'},
            columns={'three': 'peekaboo'})

data

Este comportamiento podemos hacer que sea inplace

In [ ]:
data.rename(index={'INDIANA': 'Indiana'}, inplace=True)
data

## Transformación de los datos aplicación de funciones sobre estructuras

Pandas tiene un conjunto de funciones que  permiten aplicar operaciones elemento a elemento (o fila a fila, o columna a columna) en sus estructuras de datos. 

#### Aplicación de funciones elemento a elemento sobre Series - Función map

Generamos una serie de datos

In [ ]:
serie = pd.Series([1, 2, 3, 4, 5, 6])
serie

Definimos una función

In [ ]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)

Aplicamos la función elemento a elemento

In [ ]:
serie.map(es_par)

De esta manera nos ahorramos un bucle (por lo que es más rápido)

#### Aplicación de funciones elemento a elemento sobre Dataframe - Función map

Map funciona sobre una única columna del DF. Por lo que 1º tendremos que seleccionar la columna y pasársela a Map.

Generamos un DF sobre el que trabajar

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Creamos un diccionario para aplicarlo más adelante

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Hacemos que todos los elementos de la columna food sean minúscula

In [ ]:
lowercased = data['food'].str.lower()
lowercased

Generamos una nueva columna en el DF, resultado de aplicar la serie lowercased al diccionario

In [ ]:
data['animal'] = lowercased.map(meat_to_animal)
data

Generanos una función que pasará al diccionario un elemento en minúsculas

In [ ]:
def map_fun(x):
    return meat_to_animal[x.lower()]

Aplicamos a la columna food la función, elemento a elemento

In [ ]:
data['food'].map(map_fun)

Podríamos hacerlo todo en una sola línea, pasando al map una función lambda

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

#### Aplicación de funciones elemento a elemento sobre DataFrames - Función applymap

Applymap funciona sobre todos los elementos de un DF

Generamos un DF

In [ ]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4))
dataframe

Definimos la función a aplicar

In [ ]:
def es_par(elemento):
    if elemento % 2 == 0:
        return 'Par: ' + str(elemento)
    else:
        return 'Impar: ' + str(elemento)

Map no funciona para dataframes completos (solo para una columna)

In [ ]:
dataframe.map()

Para aplicar la función a todos los elementos del DF usamos applymap

In [ ]:
dataframe.applymap(es_par)  

#### Aplicación de funciones fila a fila o columna a columna sobre DataFrames - Función apply

Generamos el DF de datos 

In [ ]:
dataframe = pd.DataFrame(np.arange(20).reshape(4, 5))
dataframe

Definimos la función a aplicar

In [ ]:
def es_suma_par(elemento):
    if np.sum(elemento) % 2 == 0:
        return 'Suma par: ' + str(np.sum(elemento))
    else:
        return 'Suma impar: ' + str(np.sum(elemento))

Aplicamos la función por columnas

In [ ]:
dataframe.apply(es_suma_par, axis=0)

Aplicamos la función por filas

In [ ]:
dataframe.apply(es_suma_par, axis=1)

### Detección y filtrado de outliers

Generamos el DF sobre el que trabajar

La función describe nos calcula los estadísticos básicos por columna

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

Queremos extraer, de la columna 2, los datos que sean mayores a 2.5

Estamos aplicando un índice booleano

In [ ]:
col = data[2]
col[np.abs(col) > 2.5]

Queremos saber qué valores del DF están por encima de 3

Obtenemos una máscara booleana

In [ ]:
cond = (np.abs(data) > 3)
cond

Comprobamos cuantos valores cumplen la condición

In [ ]:
cond.any(axis=1).sum()

Extraemos los datos que cumplen la condición

In [ ]:
data[(np.abs(data) > 3).any(axis=1)]

Acotamos los datos entre -3 y 3

np.sign devuelve el signo del elemento retornando -1 o +1

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

___
# Ejercicios

**4.3.1.** Carga el fichero  train.csv.

**4.3.2.** Calcula los estadísticos básicos de las columnas numéricas

**4.3.3.** Elimina todas las filas con NaN.

**4.3.4.** Elimina todos los registros donde la edad sea superior al tercer cuartil de esta.